In [ ]:
import tensorflow as tf
import os 
import SimpleITK as sitk 
from classification.tools.preprocessing import *

In [ ]:
trained_model_path = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/CLASSIFICATION/training/train_2/20210705-15_03_50/trained_model_20210705-15_03_50.h5'
model = tf.keras.models.load_model(trained_model_path)
model.summary()

In [ ]:
test_directory = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/CLASSIFICATION/training/train_2/images/test'
files = os.listdir(test_directory)
liste_files = []
for x in files : 
    liste_files.append(os.path.join(test_directory, x))
sorted_test = sorted(liste_files)
X_test = []
y_test = []
for i in range(0, len(sorted_test), 2):
    X_test.append(sitk.GetArrayFromImage(sitk.ReadImage(sorted_test[i+1])))
    y_test.append(get_label_from_json(sorted_test[i]))

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

print(X_test.shape)
print(y_test.shape)

In [ ]:
#evaluate
result = model.evaluate(X_test,  {'head': y_test[:,0], 
                                    'legs': y_test[:,1],
                                    'right_arm' : y_test[:,2],
                                    'left_arm' : y_test[:,3] ,
                                    })
dict(zip(model.metrics_names, result)) 

In [ ]:
#predictions
inferences = model.predict(X_test)

In [ ]:
len(inferences)
#inferences[0] head 
#inferences[1] legs 
#inferneces[2] right arm 
#inferences[3] left arm 
print(len(inferences[0][0]))

In [ ]:
from classification.tools.inference import * 
inferences_decoded = decodage_predictions(inferences)

In [ ]:
inferences_decoded[0]

In [ ]:
truth_decoded = decodage_truth(y_test)

In [ ]:
truth_decoded[0]

In [ ]:
#generate image with true/false predictions and save them 
directory = '/media/oncopole/d508267f-cc7d-45e2-ae24-9456e005a01f/CLASSIFICATION/training/train_2/inference'
affichage(X_test, inferences_decoded, truth_decoded, directory)

In [ ]:
#GENERATE MIP FROM NIFTI 

from dicom_to_cnn.model.reader.Nifti import Nifti 
from dicom_to_cnn.model.post_processing.mip.MIP_Generator import MIP_Generator

nifti_path = ''
resampled_array = Nifti(nifti_path).resample(shape_matrix=(256, 256, 1024), shape_physic=(700, 700, 2000))
resampled_array[np.where(resampled_array < 500)] = 0 #500 UH
normalize = resampled_array[:,:,:,]/np.max(resampled_array) #normalize
mip_generator = MIP_Generator(normalize)
mip = mip_generator.project(angle=0)
mip = np.expand_dims(mip, -1)
#mip of size 1024 256 1